In [1]:
import torch
import torch.nn as nn
from operators.flownet import FlowNet

In [2]:
model = FlowNet(c_in=4,
                c_hidden=[10,20,2],
                kernel=[3],
                stride=1,
                padding=1,
                batchnorm=True)
model

FlowNet(
  (models): Sequential(
    (0): Conv2d(4, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Conv2d(10, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): BatchNorm2d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): Conv2d(20, 2, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)

In [3]:
x = torch.rand(8,4,90,90)

In [4]:
model(x).shape

torch.Size([8, 2, 90, 90])

In [5]:
x = [4]